In [ ]:
import pandas as pd
import domainAPIAccess as domain
import numpy as np
import json
import os
wd_path = r'C:\dev\data\daily_scrap'

In [ ]:
QLD_postcodes = [f'{i:04}' for i in range(4149, 5000)]

for pc in QLD_postcodes:
    domain.query_API(wd_path, postcode=pc)

In [59]:
df = domain.clean_data(wd_path)
print(df.info())

test = r'C:\dev\data\main_data\domain_data_cleaned.feather'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61878 entries, 0 to 61877
Data columns (total 49 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   listingType                         61878 non-null  object 
 1   id                                  61878 non-null  int64  
 2   media                               61823 non-null  object 
 3   headline                            61875 non-null  object 
 4   summaryDescription                  61878 non-null  object 
 5   hasFloorplan                        61878 non-null  bool   
 6   hasVideo                            61878 non-null  bool   
 7   labels                              61878 non-null  object 
 8   dateListed                          61878 non-null  object 
 9   listingSlug                         61878 non-null  object 
 10  advertiser.type                     61878 non-null  object 
 11  advertiser.id                       61878

In [60]:
print(f'{len(df):,} properties succesfully saved to: {test}')

61,878 properties succesfully saved to: C:\dev\data\main_data\domain_data_cleaned.feather


In [ ]:
print(df.tail())

In [55]:
pc_df = pd.read_feather(r'C:\dev\data\checked_postcodes.feather')
unchecked_postcodes = pc_df[~pc_df['Checked']].postcode
print(f'Checking {len(pc_df) - pc_df["Checked"].sum()} postcodes')

Checking 9999 postcodes


In [ ]:
df = domain.concat_raw_data(r'C:\dev\data\NSW_first_scrap')

In [ ]:
print(f'Total rows: {len(df)}')
print(f'Number of duplicates: {np.sum(df.duplicated("id"))}')

df.drop_duplicates('id', inplace=True)

print(f'Number of unique rows: {len(df)}')

display(df.columns)
display(df.head(5))

In [ ]:
df['converted_price'] = df['priceDetails.displayPrice'].apply(domain.convert_price_to_dollars)

df['price'] = df['priceDetails.price'].fillna(df['converted_price'])

fillnavalues = {
    'propertyDetails.bathrooms': 0,
    'propertyDetails.bedrooms': 0,
    'propertyDetails.carspaces': 0
}

df.fillna(fillnavalues, inplace=True)

df.loc[(df['price'] < 25000) | (df['price'] > 10000000), 'price'] = np.nan

display(df.describe())

display(df[['listingSlug', 'priceDetails.displayPrice', 'price']].sort_values('price').head(20))

In [ ]:
df['hasPrice'] = df['price'].notna()

print(f"{df['hasPrice'].sum()} out of {len(df)} ({df['hasPrice'].sum()/len(df)*100:.2f} per cent) properties have a advertised price.")

df.reset_index(drop=True).to_feather(r'C:\dev\data\NSW_advertised_scrapped_7_January.feather')